In [5]:
#!pip install diffusers transformers accelerate safetensors gradio -q

In [ ]:
import torch
import gradio as gr
from diffusers import StableDiffusionXLPipeline
from huggingface_hub import login
from PIL import Image
import io
import datetime
import traceback
import base64

# Step 1: Login to Hugging Face
try:
    login("your_huggingface_token_here")
except Exception as e:
    print("❌ Hugging Face login failed:", e)

# Step 2: Load Model
try:
    pipe = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/sdxl-turbo",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    ).to("cuda")
except Exception as e:
    print("❌ Error loading model:", e)
    pipe = None

# Step 3: Generate image and create download link
def generate_image_with_download(prompt):
    try:
        if not prompt or len(prompt.strip()) < 5:
            raise ValueError("Prompt must be at least 5 characters.")

        if pipe is None:
            raise RuntimeError("Model not loaded. Check token or runtime.")

        image = pipe(prompt=prompt, guidance_scale=0.0, num_inference_steps=2).images[0]

        # Save image to memory
        buffer = io.BytesIO()
        image.save(buffer, format="PNG")
        buffer.seek(0)

        # Convert to base64
        img_base64 = base64.b64encode(buffer.read()).decode('utf-8')

        # Create download link
        html = f'''
        <a download="generated_image.png" href="data:image/png;base64,{img_base64}" target="_blank">
            <button style="padding: 8px 16px; background-color: #00b894; color: white; border: none; border-radius: 5px; font-weight: bold;">
                ⬇️ Download Image
            </button>
        </a>
        '''

        return image, html, "✅ Image generated successfully!"

    except Exception as e:
        traceback.print_exc()
        return None, "", f"❌ Error: {str(e)}"

# Step 4: Gradio UI
with gr.Blocks(theme=gr.themes.Soft(), css="""
#header {text-align: center; font-size: 32px; font-weight: bold; margin-bottom: 10px;}
.gr-button {font-weight: bold;}
#status {font-size: 16px; margin-top: 10px; color: #333;}
""") as demo:

    gr.HTML('<div id="header">🎨 SDXL Turbo - Text to Image Generator</div>')

    with gr.Row():
        with gr.Column(scale=1):
            prompt = gr.Textbox(label="📝 Enter your creative prompt",
                                placeholder="e.g., A futuristic dragon flying over Tokyo",
                                lines=2, max_lines=3)
            generate_btn = gr.Button("🚀 Generate Image", variant="primary")
            status_output = gr.Textbox(label="", elem_id="status", interactive=False)

        with gr.Column(scale=2):
            image_output = gr.Image(label="🖼️ Output Image", show_label=True)
            download_html = gr.HTML(label="⬇️ Click to Download")

    generate_btn.click(
        fn=generate_image_with_download,
        inputs=prompt,
        outputs=[image_output, download_html, status_output]
    )

demo.launch()


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
